In [1]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer ## Without Training Data 
import numpy as np
import pandas as pd

nltk.download('vader_lexicon') ## English Only 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\20122\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
## Load The Data
df = pd.read_csv('amazonreviews.tsv', sep='\t')
df.tail()

,label,review
9995,pos,A revelation of life in small town America in ...
9996,pos,Great biography of a very interesting journali...
9997,neg,Interesting Subject; Poor Presentation: You'd ...
9998,neg,Don't buy: The box looked used and it is obvio...
9999,pos,Beautiful Pen and Fast Delivery.: The pen was ...


In [3]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [4]:
## Check Nulls
df.isnull().sum()

label     0
review    0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
blanks = [] 

for i,lb,rv in df.itertuples():    # iterate over the DataFrame
    if type(rv) == str:            # avoid NaN values
        if rv.isspace():           # test 'review' for whitespace
            blanks.append(i)       # add matching index numbers to the list
            
blanks

In [ ]:
df.drop(blanks, inplace=True)

In [5]:
df.loc[0]['review'], df.loc[0]['label']

('Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 'pos')

In [6]:
SIA = SentimentIntensityAnalyzer()
SIA.polarity_scores(df.loc[0]['review'])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

In [7]:
for i in range (10,25) : 
    print(df.loc[i]['review'])
    print()
    print(SIA.polarity_scores(df.loc[i]['review']))
    print()
    print(df.loc[i]['label'])
    print(125*'_', '\n')

The Worst!: A complete waste of time. Typographical errors, poor grammar, and a totally pathetic plot add up to absolutely nothing. I'm embarrassed for this author and very disappointed I actually paid for this book.

{'neg': 0.36, 'neu': 0.586, 'pos': 0.054, 'compound': -0.9274}

neg
_____________________________________________________________________________________________________________________________ 

Great book: This was a great book,I just could not put it down,and could not read it fast enough. Boy what a book the twist and turns in this just keeps you guessing and wanting to know what is going to happen next. This book makes you fall in love and can heat you up,it can also make you so angery. this book can make you go throu several of your emotions. This is a quick read romance. It is something that you will want to end your day off with if you read at night.

{'neg': 0.0, 'neu': 0.836, 'pos': 0.164, 'compound': 0.9538}

pos
________________________________________________

In [8]:
df['scores'] = df['review'].apply(lambda review: SIA.polarity_scores(review))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df.tail()

,label,review,scores,compound,comp_score
9995,pos,A revelation of life in small town America in ...,"{'neg': 0.017, 'neu': 0.846, 'pos': 0.136, 'co...",0.9610,pos
9996,pos,Great biography of a very interesting journali...,"{'neg': 0.0, 'neu': 0.868, 'pos': 0.132, 'comp...",0.9544,pos
9997,neg,Interesting Subject; Poor Presentation: You'd ...,"{'neg': 0.084, 'neu': 0.754, 'pos': 0.162, 'co...",0.9102,pos
9998,neg,Don't buy: The box looked used and it is obvio...,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp...",-0.3595,neg
9999,pos,Beautiful Pen and Fast Delivery.: The pen was ...,"{'neg': 0.028, 'neu': 0.811, 'pos': 0.161, 'co...",0.9107,pos


In [9]:
## Accuracy Metrics

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

accuracy = accuracy_score(df['label'],df['comp_score'])
print("Accuracy:", accuracy, '\n')
print('Classification Report:\n', classification_report(df['label'],df['comp_score']), '\n')
print('Confusion Matrix:\n', confusion_matrix(df['label'],df['comp_score']))

Accuracy: 0.7097 

Classification Report:
               precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000
 

Confusion Matrix:
 [[2629 2468]
 [ 435 4468]]


In [10]:
a = 'This was a good movie.'
SIA.polarity_scores(a)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [11]:
a = 'This was the best, most awesome movie EVER MADE!!!'
SIA.polarity_scores(a)

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

In [12]:
a = 'This was the worst film to ever disgrace the screen.'
SIA.polarity_scores(a)

{'neg': 0.477, 'neu': 0.523, 'pos': 0.0, 'compound': -0.8074}